In [9]:
#imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import optimize

In [31]:
#reading the file
full_data = pd.read_excel("Recorded Data.xlsx")
print(pd.read_excel("Recorded Data.xlsx"))

        Nozzle Test Data      Unnamed: 1  Unnamed: 2  Unnamed: 3  Unnamed: 4  \
0   Barometric pressure:  29.35 in of Hg         NaN         NaN         NaN   
1               Test no:               1         2.0        3.00        4.00   
2         Back pressure:               0        30.0       40.00       50.00   
3        Inlet temp (C):              25        25.5       25.00       25.00   
4       Outlet temp (C):              20        19.5       22.50       22.50   
5               Orifice:            3.03         3.1        2.98        3.02   
6                      1              85        87.0       86.00       86.00   
7                      2              86        87.0       86.00       86.00   
8                      3              86        87.0       86.00       86.00   
9                      4              86        87.0       86.00       86.00   
10                     5              86        87.0       85.00       86.00   
11                     6              85

In [11]:
#reading and importing data from excel file, only displaying columns B-J (Test 1 - 9) and only rows starting with static pressures
raw_data = pd.read_excel("Recorded Data.xlsx", usecols="B:J", skiprows=6)
#print(raw_data)

## Task 1a

In [ ]:
#constants
p_t = 85 #psig given in the lab report

#creating list for normalized pressures
p_norm_data = np.zeros((len(raw_data), 9))

#calculating the normalized pressure profile
for i in range(9):
    for j in range(len(raw_data)):
        p_norm = raw_data.iloc[j,i]/p_t
        p_norm_data[j, i] = p_norm

#plotting the data
for i in range(9):
    x = np.arange(0, 30, 1)
    y = p_norm_data[:,i]
    plt.plot(x,y)
    plt.title("Test Number {} Normalized Pressure".format(i+1))
    plt.ylabel("Normalized Pressure")
    plt.xlabel("Probe Position")
    plt.show()

## Task 1b

In [ ]:
#constants
back_pressures = [0, 30, 40, 50, 60, 65, 70, 75, 80]
x = np.arange(0, 30, 1)

#finding the exit pressures for each test (at position 26)
exit_pressures = []
for i in range(9):
    exit_pressures.append(raw_data.iloc[25,i])
print(exit_pressures)

#creating the arrays for the data
p_e_norm_data = np.zeros((len(raw_data), 9))
p_b_norm_data = np.zeros((len(raw_data), 9))

#normalized exit pressures (pe/p0)
for i in range(9):
    for j in range(len(raw_data)):
        p_e_norm = exit_pressures[i]/raw_data.iloc[j,i]
        p_e_norm_data[j, i] = p_e_norm

#normalized back pressures (pb/p0)
for i in range(9):
    for j in range(len(raw_data)):
        p_b_norm = back_pressures[i]/raw_data.iloc[j,i]
        p_b_norm_data[j, i] = p_b_norm

#plotting the data
for i in range(9):
    x = p_e_norm_data[:,i]
    y = p_b_norm_data[:,i]
    plt.plot(x,y)
    plt.title("Test Number {}: Normalized Exit Pressure vs Normalied Back Pressure".format(i+1))
    plt.ylabel("Normalized Back Pressure")
    plt.xlabel("Normalized Exit Pressure")
    plt.show()

## Task 1c

In [41]:
#constants
gamma = 1.4
A_exit = np.pi*(0.2869/2)**2 #area of the nozzle at the exit, in^2
A_star = np.pi*(0.2504/2)**2 #area of the throat, in^2
A_ratio = A_exit/A_star #area ratio

#The A_ratio is equal to A/A*. Using Appendix A of "Fundamentals of Aerodynamics" By John D. Anderson the subsonic
#and sonic M numbers are determined by interpolating the data for our given A ratio
M_sub = 0.514
M_sonic = 1.673 #the sonic M number not used since at the throat in sonic conditions M=1 due to choking

#finding the pressure of the subsonic throat using the equation p/pt = (1 + (gamma-1)/2 * M**2)^(-gamma / (gamma-1))
#pt is the total pressure with p being the throat pressure
p_throat_subsonic = 85*(1 + (gamma-1)/2 * M_sub**2)**(-gamma / (gamma-1))
p_throat_sonic = 85*(1 + (gamma-1)/2 * 1**2)**(-gamma / (gamma-1))

print("Subsonic Pressure: {:.2f} psi".format(p_throat_subsonic))
print("Sonic Throat Pressure: {:.2f} psi".format(p_throat_sonic))

Subsonic Pressure: 70.98 psi
Sonic Throat Pressure: 44.90 psi


## Task 1d

In [51]:
#constants
d_orifice = 0.1625 #in
D_pipe = 3.1875 #in
A_th = np.pi*(d_orifice/2)**2 / 144 #ft^2
g_c = 32.174 #(lbm-ft)/(lbf-s^2)
F_a = 1.0 
C_D = 0.623
Beta = d_orifice/D_pipe
R = 10.731 #psi⋅ft^3⋅lbmol^−1⋅°R^−1

#calcualting rho using the equation p = rho*R*T, with p being at the throat which equals position 9
p_throat_data = [] #psig
for i in range(9):
    p_throat_data.append(raw_data.iloc[8,i])

T_recorded = []
for i in range(9):
    T_in = full_data.iloc[3,i+1] #getting the inlet recorded temp.
    T_in = T_in*(9/5)+491.67 #converting from Celsius to Rankine
    T_recorded.append(T_in)

rho_f = [] #lbm/ft^3
for i in range(9):
    rho = p_throat_data[i]/(R*T_recorded[i]) * 28.966 #going from lbmol to lbm (lbm/ft^3)
    rho_f.append(rho)
print(rho_f)

#getting delta p
delta_p = []
for i in range(9):
    p_diff = full_data.iloc[5,i+1] #in of H20
    p_diff = p_diff*5.197 #lbf/ft^2
    delta_p.append(p_diff)

#calculating m_dot
m_dot_data = []
for i in range(9):
    m_dot = A_th * F_a * (C_D / np.sqrt(1 - Beta**4)) * np.sqrt(2*g_c*rho_f[i]*delta_p[i])
    m_dot_data.append(m_dot)

print(m_dot_data) #lbm/s


[0.24142500555265115, 0.24604207858073998, 0.24142500555265115, 0.24142500555265115, 0.24142500555265115, 0.24142500555265115, 0.281662506478093, 0.3269296950192151, 0.3671671959446569]
[0.001095099066233592, 0.0011076764982521179, 0.0010860260095864485, 0.0010932904785976147, 0.0010969046718558406, 0.001100506995746114, 0.001071349307793864, 0.0010026490639625431, 0.0008985601993137457]


## Task 1e